# Using Top2Vec

In [13]:
%pip install top2vec

  Using cached top2vec-1.0.29-py3-none-any.whl (26 kB)
  Obtaining dependency information for cython<3,>=0.27 from https://files.pythonhosted.org/packages/3f/d6/9eed523aeaca42acbaa3e6d3850edae780dc7f8da9df1bf6a2ceb851839c/Cython-0.29.36-py2.py3-none-any.whl.metadata
  Using cached Cython-0.29.36-py2.py3-none-any.whl.metadata (3.1 kB)
Using cached Cython-0.29.36-py2.py3-none-any.whl (988 kB)
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.0
    Uninstalling Cython-3.0.0:
      Successfully uninstalled Cython-3.0.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install spacy

  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/d6/9e/8afc618cfed4b5dc602b11754d4d9193a268439704defae301bffca7f04c/spacy-3.6.1-cp311-cp311-win_amd64.whl.metadata
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
  Using cached spacy_loggers-1.0.4-py3-none-any.whl (11 kB)
     ---------------------------------------- 0.0/91.9 kB ? eta -:--:--
     ---------------------------------------- 91.9/91.9 kB 5.1 MB/s eta 0:00:00
  Obtaining dependency information for thinc<8.2.0,>=8.1.8 from https://files.pythonhosted.org/packages/ea/65/9fe6fe1ddb5fd34b7b81dada121e6862791e624384a2964331d0228aea38/thinc-8.1.12-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for wasabi<1.2.0,>=0.9.1 from https://files.pythonhosted.org/packages/8f/69/26cbf0bad11703241cb84d5324d868097f7a8faf2f1888354dac8883f3fc/wasabi-1.1.2-py3-none-any.whl.metadata
  Obtaining dependency information for srsly<3.0.0,>=2.4.3 from https://files.pythonhosted

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.7.1 which is incompatible.


In [7]:
import pandas as pd
from top2vec import Top2Vec
import sqlite3
from sqlite3 import Error

In [8]:
def create_connection(path):
    """ (str) -> Connection obj
    """
    connection = None
    try:
        connection = sqlite3.connect(path)
        print('Connection to SQLite DB successful!')
    except Error as e:
        print(f'The error {e} occurred.')
    return connection

In [9]:
connection = create_connection('lyrics_nlp.sqlite')
query = 'SELECT * FROM lyrics'
df = pd.read_sql(query, connection, index_col='id')
df.head()

Connection to SQLite DB successful!


,song_title,artist,album,release_year,song_lyrics
id,,,,,
1,The Kennel (Skit),DMX,...And Then There Was X,1999,\nRuff Ryders\nRide or die\nGet down or lay do...
2,One More Road to Cross,DMX,...And Then There Was X,1999,\nUh\nThis is life (This is life)\nThis is wha...
3,The Professional,DMX,...And Then There Was X,1999,\nNiggas won't creep these streets with me\n'C...
4,Fame,DMX,...And Then There Was X,1999,\nUh-huh (Y'KNOW?)\n..\nARF!\nUhh..\n\n\nTake ...
5,A Lot to Learn - Skit,DMX,...And Then There Was X,1999,"Kid: yo, yo, let me hit that. Let me hit that ..."


In [10]:
docs_bad = df['song_lyrics'].values.tolist()

In [11]:
topic_model = Top2Vec(
    docs_bad,
    embedding_model="doc2vec",
    speed="deep-learn",
    ngram_vocab=True,
    ngram_vocab_args={"connector_words": "phrases.ENGLISH_CONNECTOR_WORDS"},
)

2023-08-23 12:23:45,140 - top2vec - INFO - Pre-processing documents for training
2023-08-23 12:24:03,518 - top2vec - INFO - Creating joint document/word embedding
2023-08-23 15:12:51,455 - top2vec - INFO - Creating lower dimension embedding of documents
2023-08-23 15:13:29,516 - top2vec - INFO - Finding dense areas of documents


TypeError: 'numpy.float64' object cannot be interpreted as an integer

# BERTopic

In [1]:
# Data processing
import pandas as pd
import numpy as np

# text processing
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()
from gensim.utils import simple_preprocess

# dimension reduction
from umap import UMAP

# modeling
from bertopic import BERTopic

# Helper functions
from helper_functions import create_connection, remove_stopwords

[nltk_data] Downloading package stopwords to C:\Users\HP
[nltk_data]     USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\HP
[nltk_data]     USER\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\HP
[nltk_data]     USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
c:\Users\HP USER\anaconda3\envs\NLPenv\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\HP USER\anaconda3\envs\NLPenv\Lib\site-packages\umap\di

In [5]:
connection = create_connection('lyrics_nlp.sqlite')
query = 'SELECT * FROM lyrics'
df = pd.read_sql(query, connection, index_col='id')
df.head()

Connection to SQLite DB successful!


,song_title,artist,album,release_year,song_lyrics
id,,,,,
1,The Kennel (Skit),DMX,...And Then There Was X,1999,\nRuff Ryders\nRide or die\nGet down or lay do...
2,One More Road to Cross,DMX,...And Then There Was X,1999,\nUh\nThis is life (This is life)\nThis is wha...
3,The Professional,DMX,...And Then There Was X,1999,\nNiggas won't creep these streets with me\n'C...
4,Fame,DMX,...And Then There Was X,1999,\nUh-huh (Y'KNOW?)\n..\nARF!\nUhh..\n\n\nTake ...
5,A Lot to Learn - Skit,DMX,...And Then There Was X,1999,"Kid: yo, yo, let me hit that. Let me hit that ..."


### Text Preprocessing (optional)

In [6]:
df['lyrics_no_stop'] = df['song_lyrics'].apply(remove_stopwords)
df.head()

,song_title,artist,album,release_year,song_lyrics,lyrics_no_stop
id,,,,,,
1,The Kennel (Skit),DMX,...And Then There Was X,1999,\nRuff Ryders\nRide or die\nGet down or lay do...,ruff ryders ride die lay motherfucker rrr boy ...
2,One More Road to Cross,DMX,...And Then There Was X,1999,\nUh\nThis is life (This is life)\nThis is wha...,life life know know life life one road cross o...
3,The Professional,DMX,...And Then There Was X,1999,\nNiggas won't creep these streets with me\n'C...,niggas creep streets cause know fuckin streets...
4,Fame,DMX,...And Then There Was X,1999,\nUh-huh (Y'KNOW?)\n..\nARF!\nUhh..\n\n\nTake ...,know arf take worth birth blessing sent live d...
5,A Lot to Learn - Skit,DMX,...And Then There Was X,1999,"Kid: yo, yo, let me hit that. Let me hit that ...",kid yo yo let hit let hit yo dmx yo shorty kid...


In [7]:
df['lyrics_lemmatized'] = df['lyrics_no_stop'].apply(lambda x: ' '.join([wn.lemmatize(w) for w in x.split()]))

df.head()

,song_title,artist,album,release_year,song_lyrics,lyrics_no_stop,lyrics_lemmatized
id,,,,,,,
1,The Kennel (Skit),DMX,...And Then There Was X,1999,\nRuff Ryders\nRide or die\nGet down or lay do...,ruff ryders ride die lay motherfucker rrr boy ...,ruff ryders ride die lay motherfucker rrr boy ...
2,One More Road to Cross,DMX,...And Then There Was X,1999,\nUh\nThis is life (This is life)\nThis is wha...,life life know know life life one road cross o...,life life know know life life one road cross o...
3,The Professional,DMX,...And Then There Was X,1999,\nNiggas won't creep these streets with me\n'C...,niggas creep streets cause know fuckin streets...,nigga creep street cause know fuckin street ma...
4,Fame,DMX,...And Then There Was X,1999,\nUh-huh (Y'KNOW?)\n..\nARF!\nUhh..\n\n\nTake ...,know arf take worth birth blessing sent live d...,know arf take worth birth blessing sent live d...
5,A Lot to Learn - Skit,DMX,...And Then There Was X,1999,"Kid: yo, yo, let me hit that. Let me hit that ...",kid yo yo let hit let hit yo dmx yo shorty kid...,kid yo yo let hit let hit yo dmx yo shorty kid...


### Model Building

In [16]:
# Initiate UMAP
umap_model = UMAP(n_neighbors=15,
                    n_components=6,
                    min_dist=0,
                    metric='cosine',
                    random_state=100)

In [17]:
# initiate BERTopic
topic_model = BERTopic(umap_model=umap_model, language='english', calculate_probabilities=True)

In [18]:
topics, probabilities = topic_model.fit_transform(df['lyrics_lemmatized'])

### First Pass with settings from article.

In [10]:
# extract topics
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,6,-1_high_la_ooh_cause,"[high, la, ooh, cause, man, go, know, daaa, ba...",[hmm like care nothin man roll another blunt y...
1,0,16099,0_nigga_like_know_yeah,"[nigga, like, know, yeah, shit, bitch, go, see...",[holdin daughter booth momma somewhere bentley...
2,1,155,1_smoke_weed_high_smokin,"[smoke, weed, high, smokin, blunt, roll, one, ...",[wake cuz wake cuz homies tellin go somewhere ...
3,2,92,2_gucci_mane_like_trap,"[gucci, mane, like, trap, girl, nigga, bling, ...",[gucci please tell love main squeeze tell love...
4,3,35,3_needed_know__,"[needed, know, , , , , , , , ]","[, know, needed]"
5,4,14,4_khalifa_wiz_let_go,"[khalifa, wiz, let, go, control, see, nigga, b...",[yeah another one nigga know wiz khalifa yeah ...


In [12]:
topic_model.get_topic(1)

[('smoke', 0.09446774856735178),
 ('weed', 0.07371160390440142),
 ('high', 0.06646818907573214),
 ('smokin', 0.05055229147125472),
 ('blunt', 0.04938471488178656),
 ('roll', 0.037898881455620585),
 ('one', 0.03589003944944259),
 ('know', 0.03526070864470953),
 ('yeah', 0.034380111062123984),
 ('nigga', 0.032699281943720696)]

In [17]:
# visualize top topic keywords

topic_model.visualize_barchart(top_n_topics=5)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.023703818683868862, 0.026906759642359914,
                    0.03030839123703219, 0.04034165109018131, 0.04048059008227189],
              'xaxis': 'x',
              'y': [shit  , yeah  , know  , like  , nigga  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.04938471488178656, 0.05055229147125472, 0.06646818907573214,
                    0.07371160390440142, 0.09446774856735178],
              'xaxis': 'x2',
              'y': [blunt  , smokin  , high  , weed  , smoke  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.04250875106841323, 0.04333471095846463, 0.04373902617835908,
                    0.07271336336843999, 0.2154636654013074],
              'xaxis': 'x3',
              'y': [girl  , trap  , like  , mane  , gucci  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [1e-05, 1e-05, 1e-05, 1.397559222183218, 3.608781891561421],
              'xaxis': 'x4',
              'y': [  ,   ,   , know  , needed  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.05941437902732333, 0.07282612600414076, 0.07322018396160289,
                    0.07693804617337649, 0.08681753143712084],
              'xaxis': 'x5',
              'y': [control  , go  , let  , wiz  , khalifa  ],
              'yaxis': 'y5'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 0',
                                'x': 0.0875,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 1',
                                'x': 0.36250000000000004,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 2',
                                'x': 0.6375000000000001,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 3',
                                'x': 0.9125,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 4',
                                'x': 0.0875,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 0.4,
                 

In [18]:
topic_model.visualize_term_rank()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertext': '<b>Topic -1</b>:high_la_ooh_cause_man_go_know_daaa',
              'line': {'color': 'black', 'width': 1.5},
              'mode': 'lines+lines',
              'name': '',
              'opacity': 0.1,
              'type': 'scatter',
              'x': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
              'y': array([0.84940891, 0.27320821, 0.14921729, 0.13127455, 0.11822627, 0.11481683,
                          0.09335366, 0.07377304, 0.07377304, 0.06861764])},
             {'hovertext': '<b>Topic 0</b>:nigga_like_know_yeah_shit_bitch_go_',
              'line': {'color': 'black', 'width': 1.5},
              'mode': 'lines+lines',
              'name': '',
              'opacity': 0.1,
              'type': 'scatter',
              'x': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
              'y': array([0.04048059, 0.04034165, 0.03030839, 0.02690676, 0.02370382, 0.023208  ,
                          0.02187829, 0.02156687, 0.02129254, 0.02011867])},
             {'hovertext': '<b>Topic 1</b>:smoke_weed_high_smokin_blunt_roll_o',
              'line': {'color': 'black', 'width': 1.5},
              'mode': 'lines+lines',
              'name': '',
              'opacity': 0.1,
              'type': 'scatter',
              'x': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
              'y': array([0.09446775, 0.0737116 , 0.06646819, 0.05055229, 0.04938471, 0.03789888,
                          0.03589004, 0.03526071, 0.03438011, 0.03269928])},
             {'hovertext': '<b>Topic 2</b>:gucci_mane_like_trap_girl_nigga_bli',
              'line': {'color': 'black', 'width': 1.5},
              'mode': 'lines+lines',
              'name': '',
              'opacity': 0.1,
              'type': 'scatter',
              'x': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
              'y': array([0.21546367, 0.07271336, 0.04373903, 0.04333471, 0.04250875, 0.03714562,
                          0.03660396, 0.03463047, 0.03459526, 0.02765809])},
             {'hovertext': '<b>Topic 4</b>:khalifa_wiz_let_go_control_see_nigg',
              'line': {'color': 'black', 'width': 1.5},
              'mode': 'lines+lines',
              'name': '',
              'opacity': 0.1,
              'type': 'scatter',
              'x': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
              'y': array([0.08681753, 0.07693805, 0.07322018, 0.07282613, 0.05941438, 0.0564365 ,
                          0.05519711, 0.05453586, 0.05361475, 0.04960526])}],
    'layout': {'height': 500,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'showlegend': False,
               'template': '...',
               'title': {'font': {'color': 'Black', 'size': 22},
                         'text': '<b>Term score decline per Topic</b>',
                         'x': 0.5,
                         'xanchor': 'center',
                         'y': 0.9,
                         'yanchor': 'top'},
               'width': 800,
               'xaxis': {'dtick': 2, 'range': [0, 10], 'tick0': 1, 'title': {'text': 'Term Rank'}},
               'yaxis': {'title': {'text': 'c-TF-IDF score'}}}
})

In [20]:
topic_model.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'nigga | like | know | yeah | shit', 16099],
                                   [1, 'smoke | weed | high | smokin | blunt', 155],
                                   [2, 'gucci | mane | like | trap | girl', 92],
                                   [3, 'needed | know |  |  | ', 35],
                                   [4, 'khalifa | wiz | let | go | control', 14]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': array([16099,   155,    92,    35,    14], dtype=int64),
                         'sizemode': 'area',
                         'sizeref': 10.061875,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([-1.9978615, -1.2851299, -1.4602382, -1.6748097, -2.5845845],
                         dtype=float32),
              'xaxis': 'x',
              'y': array([12.4518385, 13.170892 , 11.923066 , 13.688639 , 11.83092  ],
                         dtype=float32),
              'yaxis': 'y'}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'D1',
                                'x': -2.9722721457481383,
                                'y': 12.899108338356019,
                                'yshift': 10},
                               {'showarrow': False,
                                'text': 'D2',
                                'x': -2.032316282391548,
                                'xshift': 10,
                                'y': 15.741934490203857}],
               'height': 650,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'itemsizing': 'constant', 'tracegroupgap': 0},
               'margin': {'t': 60},
               'shapes': [{'line': {'color': '#CFD8DC', 'width': 2},
                           'type': 'line',
                           'x0': -2.032316282391548,
                           'x1': -2.032316282391548,
                           'y0': 10.056282186508179,
                           'y1': 15.741934490203857},
                          {'line': {'color': '#9E9E9E', 'width': 2},
                           'type': 'line',
                           'x0': -2.9722721457481383,
                           'x1': -1.0923604190349578,
                           'y0': 12.899108338356019,
                           'y1': 12.899108338356019}],
               'sliders': [{'active': 0,
                            'pad': {'t': 50},
                            'steps': [{'args': [{'marker.color': [['red', '#B0BEC5', '#B0BEC5', '#B0BEC5', '#B0BEC5']]}],
                                       'label': 'Topic 0',
                                       'method': 'update'},
                                      {'args': [{'marker.color': [['#B0BEC5', 'red', '#B0BEC5', '#B0BEC5', '#B0BEC5']]}],
                                       'label': 'Topic 1',
                                       'method': 'update'},
                                      {'args': [{'marker.color': [['#B0BEC5', '#B0BEC5', 'red', '#B0BEC5', '#B0BEC5']]}],
                                       'label': 'Topic 2',
                                       'method': 'update'},
                                      {'args': [{'marker.color': [['#B0BEC5', '#B0BEC5', '#B0BEC5', 'red', '#B0BEC5']]}],
                                       'label': 'Topic 3',
                                       'method': 'update'},
                                      {'args': [{'marker.color': [['#B0BEC5', '#B0BEC5', '#B0BEC5', '#B0BEC5', 'red']]}],
                             

In [21]:
topic_model.visualize_hierarchy()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': array([0.        , 0.35780642, 0.35780642, 0.        ]),
              'xaxis': 'x',
              'y': array([-25., -25., -35., -35.]),
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': array([0.        , 0.52743531, 0.52743531, 0.35780642]),
              'xaxis': 'x',
              'y': array([-15., -15., -30., -30.]),
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': array([0.52743531, 0.71742529, 0.71742529, 0.        ]),
              'xaxis': 'x',
              'y': array([-22.5, -22.5, -45. , -45. ]),
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(0,116,217)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': array([0.        , 1.11542626, 1.11542626, 0.71742529]),
              'xaxis': 'x',
              'y': array([ -5.  ,  -5.  , -33.75, -33.75]),
              'yaxis': 'y'}],
    'layout': {'autosize': False,
               'height': 275,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'hovermode': 'closest',
               'plot_bgcolor': '#ECEFF1',
               'showlegend': False,
               'template': '...',
               'title': {'font': {'color': 'Black', 'size': 22},
                         'text': '<b>Hierarchical Clustering</b>',
                         'x': 0.5,
                         'xanchor': 'center',
                         'yanchor': 'top'},
               'width': 1000,
               'xaxis': {'mirror': 'allticks',
                         'rangemode': 'tozero',
                         'showgrid': False,
                         'showline': True,
                         'showticklabels': True,
                         'ticks': 'outside',
                         'type': 'linear',
                         'zeroline': False},
               'yaxis': {'mirror': 'allticks',
                         'range': [-50.0, 0.0],
                         'rangemode': 'tozero',
                         'showgrid': False,
                         'showline': True,
                         'showticklabels': True,
                         'tickmode': 'array',
                         'ticks': 'outside',
                         'ticktext': [3_needed_know_, 4_khalifa_wiz_let,
                                      0_nigga_like_know, 1_smoke_weed_high,
                                      2_gucci_mane_like],
                         'tickvals': [-5.0, -15.0, -25.0, -35.0, -45.0],
                         'type': 'linear',
                         'zeroline': False}}
})

In [22]:
topic_model.visualize_heatmap()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>Similarity Score: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': [0_nigga_like_know, 1_smoke_weed_high, 2_gucci_mane_like,
                    3_needed_know_, 4_khalifa_wiz_let],
              'xaxis': 'x',
              'y': [0_nigga_like_know, 1_smoke_weed_high, 2_gucci_mane_like,
                    3_needed_know_, 4_khalifa_wiz_let],
              'yaxis': 'y',
              'z': array([[0.9999999 , 0.9193097 , 0.90216005, 0.11187288, 0.85075307],
                          [0.9193097 , 0.99999964, 0.82149816, 0.12237155, 0.80182916],
                          [0.90216005, 0.82149816, 1.        , 0.10640633, 0.7711693 ],
                          [0.11187288, 0.12237155, 0.10640633, 1.        , 0.09187127],
                          [0.85075307, 0.80182916, 0.7711693 , 0.09187127, 1.        ]],
                         dtype=float32)}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Similarity Score'}},
                             'colorscale': [[0.0, 'rgb(247,252,240)'], [0.125,
                                            'rgb(224,243,219)'], [0.25,
                                            'rgb(204,235,197)'], [0.375,
                                            'rgb(168,221,181)'], [0.5,
                                            'rgb(123,204,196)'], [0.625,
                                            'rgb(78,179,211)'], [0.75,
                                            'rgb(43,140,190)'], [0.875,
                                            'rgb(8,104,172)'], [1.0,
                                            'rgb(8,64,129)']]},
               'height': 800,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'title': {'text': 'Trend'}},
               'margin': {'t': 60},
               'showlegend': True,
               'template': '...',
               'title': {'font': {'color': 'Black', 'size': 22},
                         'text': '<b>Similarity Matrix</b>',
                         'x': 0.55,
                         'xanchor': 'center',
                         'y': 0.95,
                         'yanchor': 'top'},
               'width': 800,
               'xaxis': {'anchor': 'y', 'constrain': 'domain', 'domain': [0.0, 1.0], 'scaleanchor': 'y'},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'constrain': 'domain', 'domain': [0.0, 1.0]}}
})

### Second pass with n_components increased from 5 to 10
Higher dimensionality for UMAP clustering

In [11]:
# extract topics
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12011,-1_know_nigga_shit_bitch,"[know, nigga, shit, bitch, go, see, cause, bac...",[pimp ho pimp ho hort rap around rocked beat s...
1,0,748,0_nigga_fuck_shit_bitch,"[nigga, fuck, shit, bitch, as, ya, know, real,...",[ayy rero happenin look window see fucked nigg...
2,1,295,1_album_man_know_um,"[album, man, know, um, call, ticket, yo, wycle...",[yo um lady say real estate tellin sayin wanna...
3,2,191,2_love_baby_girl_know,"[love, baby, girl, know, feel, need, make, wan...",[madness mind never really knew love time boy ...
4,3,150,3_smoke_weed_blunt_high,"[smoke, weed, blunt, high, smokin, roll, bud, ...",[wake early morning feel good smoking shit wis...
...,...,...,...,...,...
88,87,10,87_hunnids_blue_money_asleep,"[hunnids, blue, money, asleep, skrt, preach, p...",[work wit wanna thank fa comin tonight dis iz ...
89,88,10,88_world_bomb_president_people,"[world, bomb, president, people, although, pic...",[close mind close eye see heart forgive murder...
90,89,10,89_woah_thou_sunshine_tweakin,"[woah, thou, sunshine, tweakin, wow, flyest, k...",[peace king peace king listen wrote book life ...
91,90,10,90_sw_swinging_pocket_super,"[sw, swinging, pocket, super, fat, alright, po...",[yea vibe crazy right lean super power dive pu...


In [15]:
# topics based on n_components = 8
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,11615,-1_nigga_know_shit_bitch,"[nigga, know, shit, bitch, fuck, see, go, caus...",[bruh yo nah sayin somethin guess met last nig...
1,0,1321,0_rock_rhyme_mc_beat,"[rock, rhyme, mc, beat, hip, cause, mic, one, ...",[rap go straight top keep sure dropped trick n...
2,1,192,1_nigga_fuck_shit_bitch,"[nigga, fuck, shit, bitch, as, real, fuckin, y...",[brother insist usin word nigga know bringin b...
3,2,175,2_girl_ody_sexy_hot,"[girl, ody, sexy, hot, let, dick, sex, baby, t...",[sexy lady walkin around downtown shoppin mall...
4,3,168,3_album_man_um_music,"[album, man, um, music, know, soul, eminem, mi...",[check look go love hate pleasure pain four al...
...,...,...,...,...,...
79,78,11,78_hotep_unpredictable_yet_police,"[hotep, unpredictable, yet, police, ranging, n...",[yo woke darkness shot siren look window peep ...
80,79,11,79_regulation_connect_dot_watch,"[regulation, connect, dot, watch, chaser, brow...",[mask face nacho libre grand theft auto call c...
81,80,11,80_feelin_project_unleaded_ceilin,"[feelin, project, unleaded, ceilin, tank, buil...",[world run mill name furl owner building stone...
82,81,11,81_demon_atom_angel_demand,"[demon, atom, angel, demand, god, grotesque, h...",[atom waiting god excuse atom sittin telling p...


In [19]:
# topics based on n_components = 6
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,11164,-1_nigga_know_bitch_shit,"[nigga, know, bitch, shit, fuck, go, see, caus...",[ayo voice raspy fuck cause thirty one hahaha ...
1,0,1551,0_love_know_god_life,"[love, know, god, life, feel, time, never, one...",[wanna say wanna say feel look mirror know han...
2,1,616,1_rock_rhyme_mc_hip,"[rock, rhyme, mc, hip, beat, hop, mic, cause, ...",[mic one reason heard microphone hope miss wor...
3,2,589,2_nigga_fuck_shit_bitch,"[nigga, fuck, shit, bitch, as, ya, know, real,...",[flip flop as nigga ah geto boy outlawz recogn...
4,3,141,3_smoke_high_weed_blunt,"[smoke, high, weed, blunt, smokin, roll, bud, ...",[aw last song last motherfuckin one time light...
...,...,...,...,...,...
68,67,11,67_summertime_summer_hey_love,"[summertime, summer, hey, love, feel, everywhe...",[drum please summer summer summertime time sit...
69,68,10,68_bye_hov_incredible_baby,"[bye, hov, incredible, baby, love, diamond, wo...",[baby hit weed might calm rub foot listenin ev...
70,69,10,69_wake_playa_hater_interrupt,"[wake, playa, hater, interrupt, vibrate, kicke...",[way trying disrespect sort house club music g...
71,70,10,70_spree_life_killing_tryna,"[spree, life, killing, tryna, titty, weed, pin...",[durkio told positive shit lately wanna show b...
